In [11]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [1]:
!ls
!nvidia-smi

sample_data
Tue Dec 23 19:11:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------

In [2]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 332 (delta 122), reused 123 (delta 58), pack-reused 138 (from 1)
Receiving objects: 100% (332/332), 98.11 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (191/191), done.
/content/no-time-to-backprop


In [3]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-linux-cuda.txt  run_all_toy.sh
hpt.py				 requirements-macos-cpu.txt   run_on_gpu.ipynb
plotting_scripts		 requirements.mac.txt	      run_toy_task.py
requirements-base.txt		 requirements.txt	      test_rec.py
requirements.from_linux_env.txt  run_all_benchmarks.sh	      tgap


In [4]:
%pip install -r requirements-linux-cuda.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.2/488.2 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 100.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 101.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 123.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 25.9 MB/s eta 0:00:00

In [5]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [7]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
object address  : 0x7bd2cbad2f20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [6]:
# Optuna trials
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --batching_strategy rearranged --num_epochs 100 --dedupe --hpt optuna --hpt_config ./tgap/benchmarks/hpt_config_example.json


[*] Results will be stored in /content/no-time-to-backprop/results_for_benchmarks/mooc_ONLINE
[*] d_model set to 32
[I 2025-12-23 19:12:45,913] A new study created in memory with name: no-name-a4e93f13-99bf-4d22-b241-d154c6f06ddb

[*] ----------------------------- Iteration: 0 -----------------------------

[*] Trial: <optuna.trial._trial.Trial object at 0x7801fbc07920>
[*] HPT: {'memory': 5, 'seed': 43, 'num_layers': 4, 'state_size': 50, 'd_model_factor': 1.0, 'dropout': 0.07326237754544744, 'pos_weight': 1.8862423347191157, 'batch_size': 128, 'n_accumulation_steps': 11, 'learning_rate': 0.00010255775205731647, 'rec_learning_factor': 0.5, 'beta1': 0.8835860676854593, 'beta2': 0.983366232402884, 'weight_decay': 0.035834535831831535}
[*] Npz file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists with matching meta. Using existing file.
W1223 19:12:46.170364     684 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support

In [1]:
!ls

sample_data
